# HDF5 format for PMaps for Gate --> HDF5 converter

In [6]:
import tables

In GATE, the PMaps are stored using the AddHitMap method of the GATE event.

Each hit map has several properties which are set upon creation and can be accessed with Get methods, including:
- GetTimeMap: a vector of maps containing sensor amplitudes for sensors with nonzero values - std::vector$<$std::map$<$int,float$>>$
- GetStartTime: a float value indicating the time at which the initial map in TimeMap was recorded
- GetEndTime: a float value indicating the time at which the final map in TimeMap was recorded
- GetTimeSample: a float value indicating the amount of time elapsed between maps in TimeMap
- GetSignalType: a integer value of 1 for S1 and 2 for S2

The hit maps are obtained in Centella by calling GetHitMaps(gate.SIPM) for anode maps and GetHitMaps(gate.PMT) for cathode maps.

The proposed method of storage in HDF5 is to organize the data into rows in a Pytable corresponding to individual peaks. As the individual PMT information is not so important as the sum, only the sum from the cathode plane is saved. S1 peaks are saved (with all 0s in the anode) for the purposes of keeping t0.  The PMap table is defined as follows:

In [11]:
# Anode PMaps
class PMap(tables.IsDescription):
    """
    Stores the PMaps
    """
    event_indx = tables.Int16Col(pos=1)
    signal_type = tables.Int16Col(pos=2)
    time = tables.Float32Col(pos=3)
    cathode_sum = tables.Float32Col(pos=4)
    ID_list = tables.Int16Col(shape=1792, pos=5)
    amp_list = tables.Float32Col(shape=1792, pos=6)

Note that the class allows for 1792 sensor values.  It may not be necessary to allow for so many SiPM sensor values since it is unlikely that all will be lit up for a single event.  If PyTables essentially ignores trailing zeros, it may actually be better to use the same class type for both groups (or even just make 1 group with an additional flag specifying anode or cathode) with 1792 IDs and amplitudes.

The output HDF5 PyTable will then have rows corresponding to anode TimeMaps + the corresponding cathode sum (so several rows for a single event in each group):

[evt 0] [S1] [time 0] [cathode_sum] [ID_list:0-11 0] [amp_list:0-1791 0]<br>
[evt 0] [S1] [time 1] [cathode_sum] [ID_list:0-11 1] [amp_list:0-1791 1]<br>
[evt 0] [S1] [time 2] [cathode_sum] [ID_list:0-11 2] [amp_list:0-1791 2]<br>
...<br>
[evt 0] [S2_0] [time 0] [cathode_sum] [ID_list:0-11 0] [amp_list:0-1791 0]<br>
[evt 0] [S2_0] [time 1] [cathode_sum] [ID_list:0-11 1] [amp_list:0-1791 1]<br>
...<br>
[evt 0] [S2_1] [time 0] [cathode_sum] [ID_list:0-11 0] [amp_list:0-1791 0]<br>
...<br>
[evt 1] [S1_or_S2] [cathode_sum] [time 0] [ID_list:0-11 0] [amp_list:0-1791 0]<br>
[evt 1] [S1_or_S2] [cathode_sum] [time 1] [ID_list:0-11 1] [amp_list:0-1791 1]<br>
[evt 1] [S1_or_S2] [cathode_sum] [time 2] [ID_list:0-11 2] [amp_list:0-1791 2]<br>
... ...

where in each case the time is calculated as the (StartTime of the PMap) + (TimeMap number) * TimeSample.